In [4]:
from subprocess import check_call, STDOUT
import os
from astropy.io import fits
import numpy.polynomial.polynomial as poly
from numpy.polynomial import Polynomial as P
import tempfile
from datetime import datetime
import shutil, shlex
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas
from functools import reduce
from scipy.signal import butter,filtfilt

%matplotlib widget

In [2]:
# Inspect noise file
plt.close()
noise_sims = "noise/noise_chan1.fits"
f = fits.open(noise_sims)
ADCdata = f["TESRECORDS"].data['ADC']
print("ADC data dims=", ADCdata.shape)
ADCmax = np.amax(ADCdata, axis=1)
ADCmean = np.mean(ADCdata, axis=1)
ADCstd = np.std(ADCdata, axis=1)

noiseData = pandas.DataFrame ({'MEANNOISE': ADCmean, 'STDNOISE': ADCstd}, columns = ['MEANNOISE','STDNOISE'])
mean_std = np.mean(ADCstd)
sigma_std = np.std(ADCstd)
mean_base = np.mean(ADCmean)
sigma_base = np.std(ADCmean)

print("Mean STDNOISE=", mean_std)
print("STD of STDNOISE=", sigma_std)
print("Mean BASELINE=", mean_base)
print("STD of BASELINE=", sigma_base)

fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(1, 3, 1)
bin_heights, bin_borders, _ = ax1.hist(ADCmax, bins=20, alpha=0.4)
ax1.set_title("Distribution of Max SDC values")
ax1.set_ylabel("# records")
ax1.set_xlabel("ADC (a.u.)")
ax2 = fig.add_subplot(1, 3, 2)
bin_heights, bin_borders, _ = ax2.hist(ADCmean, bins=20, alpha=0.4)
ax2.set_title("Distribution of Mean ADC values")
ax2.set_ylabel("# records")
ax2.set_xlabel("ADC (a.u.)")
ax2.axvline(mean_base, ls="--", color="gray", alpha=0.5)
ax2.axvline(mean_base-sigma_base, ls="--", color="gray", alpha=0.5)
ax2.axvline(mean_base+sigma_base, ls="--", color="gray", alpha=0.5)
ax3 = fig.add_subplot(1, 3, 3)
bin_heights, bin_borders, _ = ax3.hist(ADCstd, bins=20, alpha=0.4)
ax3.set_title("Distribution of STD ADC values")
ax3.set_ylabel("# records")
ax3.set_xlabel("ADC (a.u.)")
ax3.axvline(mean_std, ls="--", color="gray", alpha=0.5)
ax3.axvline(mean_std-sigma_std, ls="--", color="gray", alpha=0.5)
ax3.axvline(mean_std+sigma_std, ls="--", color="gray", alpha=0.5)
fig.tight_layout()




ADC data dims= (1834, 8192)
Mean STDNOISE= 8.738369
STD of STDNOISE= 0.12467109
Mean BASELINE= 8344.179
STD of BASELINE= 0.8833343


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# lowpass filtering
#import plotly.graph_objects as go
rec_length = 8192
fs = 195312.5      # sample rate, Hz
T = rec_length/fs  # Sample Period (s)
cutoff = 5000      # desired cutoff frequency of the filter, Hz 
nyq = 0.5 * fs     # Nyquist Frequency
order = 8          # sin wave can be approx represented as quadratic
n = rec_length     # total number of samples

def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

y = butter_lowpass_filter(ADCdata[0], cutoff, fs, order)

fig = plt.figure(figsize=(6,3))
ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(range(n), ADCdata[0], label="signal")
ax1.plot(range(n), y, label="filterd signal")
ax1.legend()
#fig = go.Figure()
#fig.add_trace(go.Scatter(
#            y = data,
#            line =  dict(shape =  'spline' ),
#            name = 'signal with noise'
#            ))
#fig.add_trace(go.Scatter(
#            y = y,
#            line =  dict(shape =  'spline' ),
#            name = 'filtered signal'
#            ))
#fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
plt.close()

def nrecs(data, val):
    nn = np.zeros(len(val))
    for i in range(len(val)):
        nn[i] = len(data[data>val[i]])
    return nn
        
fig = plt.figure(figsize=(10,8))
ax1 = fig.add_subplot(2,2,1)
thres = 1.5*sigma_std
devs = abs(ADCstd-mean_std)
nrecordss = nrecs(devs,devs)
ax1.plot(nrecordss, devs, ls="",marker=".", color="tab:green" )
ax1.axhline(sigma_std, ls="--", color="tab:red")
ax1.text(1200, sigma_std, "1*SIGMA_STDNOISE")
ax1.axhline(thres, ls="--", color="tab:red")
ax1.text(1200, thres, "1.5*SIGMA_STDNOISE")
ax1.fill_between(nrecordss, min(devs), max(devs), where=devs > thres, color='gray', alpha=0.05)
ax1.set_xlabel("Number of records Deviation> Deviation")
ax1.set_ylabel("Deviation = abs(STD - <STD>)")
title = "<STD>=" + '{:0.2f}'.format(mean_std) + ";   SIGMA(STD)=" + '{:0.2f}'.format(sigma_std)
ax1.set_title(title)
ints_to_remove_1 = np.where(devs>thres)[0]
print("Using STDNOISE:")
print("Number of intervals > thres to remove:", len(ints_to_remove_1))
print("Intervals to remove:", ints_to_remove_1)

ax2 = fig.add_subplot(2,2,2)
devm = abs(ADCmean - mean_base)
nrecordsm = nrecs(devm, devm)
ax2.plot(nrecordsm, devm, ls="",marker=".", color="tab:green" )
thres = 1.5*sigma_base
ax2.axhline(thres, ls="--", color="tab:red")
ax2.fill_between(nrecordsm, min(devm), max(devm), where=devm > thres, color='gray', alpha=0.05)
ax2.axhline(sigma_base, ls="--", color="tab:red")
ax2.text(1250, sigma_base, "1*SIGMA(baseline)")
ax2.axhline(thres, ls="--", color="tab:red")
ax2.text(1250, thres, "1.5*SIGMA(baseline)")
ax2.set_xlabel("Number of records Deviation> Deviation")
ax2.set_ylabel("Deviation = abs(BASELINE - <BASELINE>)")
#title = "Deviation from mean NOISE BASELINE (" + str(mean_base) + ")"
title = "<BASELINE>=" + '{:0.2f}'.format(mean_base) + ";   SIGMA(BASELINE)=" + '{:0.2f}'.format(sigma_base)
ax2.set_title(title)
print("Using MEANNOISE Dev:")
ints_to_remove_2 = np.where(devm>thres)[0]
print("Number of intervals > thres to remove:", len(ints_to_remove_2))
print("Intervals to remove:", ints_to_remove_2)

ax3 = fig.add_subplot(2,2,3)
ax3.plot(ints_to_remove_1, len(ints_to_remove_1)*[0.5], color="tab:green", 
         marker="o", ls="", alpha=0.5,fillstyle='none', label="Records STD")
ax3.plot(ints_to_remove_2, len(ints_to_remove_2)*[1], color="tab:blue", 
         marker="x", ls="", alpha=0.5, label="Records BASELINE")
ax3.set_xlabel("Record number")
#ax4.plot(nrecordss, devs/max(devs), ls="",marker="." , label="Deviation: std")
#ax4.plot(nrecordsm, devm/max(devm), ls="",marker="." , label="Deviation: mean")
#ax4.axvline(167, ls="--", color="tab:red")
#thres = devm[(np.abs(nrecordsm - 167)).argmin()]/max(devm)
#ax4.axhline(thres, ls="--", color="tab:red")
#ax4.set_xlabel("Number of records Deviation> Deviation")
#ax4.set_ylabel("Deviation")
#ax4.legend()
#ydata = np.intersect1d(np.array(ints_to_remove_1), np.array(ints_to_remove_2))
#ax3.plot(ydata, len(ydata)*[0.5], label="STDdev+ MEANdev", ls="", marker=".", fillstyle='none')
ax3.set_ylim(0,2)
ax3.legend()

ax4 = fig.add_subplot(2,2,4)
ydata = np.intersect1d(np.array(ints_to_remove_1), np.array(ints_to_remove_2))
ax4.plot(ydata, len(ydata)*[0.5], ls="", marker=".", fillstyle='none', label="In STD && BASELINE criteria", color="magenta")
ydata2 = np.setdiff1d(np.array(ints_to_remove_1),np.array(ints_to_remove_2))
ax4.plot(ydata2, len(ydata2)*[1], ls="", marker=".", fillstyle='none', label="Only in STD || BASELINE criteria")
ax4.set_xlabel("Record number")
ax4.set_ylim(0,2)
ax4.legend()

fig.tight_layout()
display(noiseData)


NameError: name 'plt' is not defined